<a href="https://colab.research.google.com/github/lclazx/nlp_learning/blob/master/%E2%80%9Cextract_data_with_bert_hub_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 步骤一：引用所需资源

In [2]:
!pip install bert-tensorflow

import tensorflow as tf
import tensorflow_hub as hub
import bert
from bert import optimization
from bert import tokenization
import tf_metrics


步骤二：准备数据

In [0]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 input_ids,
                 input_mask,
                 segment_ids,
                 token_label_ids,
                 predicate_label_id,
                 is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.token_label_ids = token_label_ids
        self.predicate_label_id = predicate_label_id
        self.is_real_example = is_real_example

In [0]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

In [0]:
def convert_single_example(ex_index, example, token_label_list, predicate_label_list, max_seq_length, tokenizer):
  if isinstance(example, PaddingInputExample):
    return InputFeatures(
        input_ids=[0]*max_seq_length,
        input_mask=[0]*max_seq_length,
        segment_ids=[0]*max_seq_length,
        token_label_ids=[0]*max_seq_length,
        predicate_label_id=[0],
        is_real_example=False
    )
  token_label_map={}
  for(i, label) in enumerate(token_label_list):
    token_label_map[label]=i
  predicate_label_map={}
  for(i, label) in enumerate(predicate_label_list):
    predicate_label_map[label] = i
   
  text_token = example.text_token[0:-1]
  text_predicate = example.text_token[-1]
  token_label = example.token_label[0:-1]
  token_predicate = example.token_label[-1]
  assert token_predicate == text_predicate

  tokens_b = [text_predicate]*len(text_token)
  predicate_id = predicate_label_map[text_predicate]

  _truncate_seq_pair(text_token, tokens_b, max_seq_length-3)

  tokens = []
  token_label_ids = []
  segment_ids = []
  tokens.append('[CLS]')
  token_label_ids.append(token_label_map["[CLS]"])
  segment_ids.append(0)

  for token, label in zip(text_token, token_label):
    tokens.append(token)
    token_label_ids.append(token_label_map[label])
    segment_ids.append(0)

  tokens.append('[SEP]')
  token_label_ids.append(token_label_map['[SEP]'])
  segment_ids.append(0)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)
  bias = 1
  for token in tokens_b:
    input_ids.append(predicate_id + bias)
    segment_ids.append(1)
    token_label_ids.append(token_label_map['[category]'])

  input_ids.append(tokenizer.convert_tokens_to_ids(['[SEP]'])[0])
  segment_ids.append(1)
  token_label_ids.append(token_label_map['[SEP]'])

  input_mask = [1]*len(input_ids)
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)
    token_label_ids.append(0)
    tokens.append('[Padding]')

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length
  assert len(token_label_ids) == max_seq_length

  if ex_index<5:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid:%s" % (example.guid))
    tf.logging.info("tokens: %s" % ''.join([tokenization.printable_text(x) for x in tokens]))
    tf.logging.info('input_ids: %s' % ' '.join([str(x) for x in input_ids]))
    tf.logging.info('input_mask: %s' % ' '.join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % ' '.join([str(x) for x in segment_ids]))
    tf.logging.info('token_label_ids: %s' % ' '.join([str(x) for x in token_label_ids]))
    tf.logging.info('predicate_id: %s' % str(predicate_id))

  feature = InputFeatures(
      input_ids = input_ids,
      input_mask = input_mask,
      segment_ids = segment_ids,
      token_label_ids = token_label_ids,
      predicate_label_id = [predicate_id],
      is_real_example=True
  )

  return feature
  

In [0]:
def convert_examples_to_features(examples, token_label_list, predicate_label_list, max_seq_length, tokenizer):
  features = []
  for (index, example) in enumerate(examples):
    feature = convert_single_example(index, example, token_label_list, predicate_label_list, max_seq_length, tokenizer)
    features.append(feature)
  return features


In [0]:
class InputExample(object):
  def __init__(self, guid, text_token, token_labels):
    self.guid=guid
    self.text_token=text_token
    self.token_label=token_labels

class PaddingInputExample(object):
  """Truncates a sequence pair in place to the maximum length."""


In [0]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

OUTPUT_DIR = 'extract_training_output' #@param
DO_DELETE =  False#@param
USE_BUCKET = True #@param
BUCKET = 'bert_classification'#@param

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  
if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
  

In [9]:
from apiclient.http import MediaIoBaseDownload
def download_file(output_dir, source_file):
  with open(output_dir, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=BUCKET, object=source_file)
    media = MediaIoBaseDownload(f, request)
    done = False
    while not done:
      _, done = media.next_chunk()

download_file(output_dir='/tmp/train_data.json', source_file='raw_data/train_data.json/train_data.json')
download_file(output_dir='/tmp/test_data.json', source_file='raw_data/test_data_postag.json/test_data_postag.json')
download_file(output_dir='/tmp/dev_data.json', source_file='raw_data/dev_data.json/dev_data.json')

print('Downloaded')

Downloaded


In [10]:
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1'
TRAINABLE = True

def create_tokenizer_from_hub_module():
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature='tokenization_info', as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info['vocab_file'], tokenization_info['do_lower_case']])
    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
def find_bio_in_text_token(so_object_text, text_token):
  for i, token in enumerate(text_token):
    trimmed_token = token.replace('##', '')
    if so_object_text.startswith(trimmed_token):
      for j in range(i+1,len(text_token)):
        concatenated_token = ''.join([x.replace("##",'') for x in text_token[i:j]])
        if so_object_text == concatenated_token:
          return i, j
        elif so_object_text.startswith(concatenated_token):
          continue
        else:
          break
  return -1,-1

In [0]:
def build_SPO_Example(data):
  text = data['text']
  spo_list = data['spo_list']
  text_token = tokenizer.tokenize(text)
  examples = []
  for spo in spo_list:
    subject = spo['subject']
    obj = spo['object']
    predicate = spo['predicate']
    subject_start_index, subject_end_index = find_bio_in_text_token(subject, text_token)
    obj_start_index, obj_end_index = find_bio_in_text_token(obj, text_token)
    text_token_with_prediction=[]
    for token in text_token:
      text_token_with_prediction.append(token)
    token_labels = ['O']*len(text_token_with_prediction)
    text_token_with_prediction.append(predicate)
    token_labels.append(predicate)

    if subject_start_index>-1:
      token_labels[subject_start_index] = 'B-SUB'
      for i in range(subject_start_index+1, subject_end_index):
        token_labels[i] = 'I-SUB'
    
    if obj_start_index > -1:
      token_labels[obj_start_index] = 'B-OBJ'
      for i in range(obj_start_index+1, obj_end_index):
        token_labels[i] = 'I-OBJ'

    #examples.append({'text_token': text_token_with_prediction, 'token_labels': token_labels})
    example = InputExample(guid=None, text_token=text_token_with_prediction, token_labels=token_labels)
    examples.append(example)
  return examples
      


In [13]:
test_data = {'text': '查尔斯·阿兰基斯（Charles Aránguiz），1989年4月17日出生于智利圣地亚哥，智利职业足球运动员，司职中场，效力于德国足球甲级联赛勒沃库森足球俱乐部',
             'spo_list':[
                       {
                          'predicate':'出生地',
                          'object':'圣地亚哥',
                          'subject':'查尔斯·阿兰基斯'
                       },
                       {
                          'predicate':'出生日期',
                          'object':'1989年4月17日',
                          'subject':'查尔斯·阿兰基斯'

                       }
                  ]
        }

examples = build_SPO_Example(test_data)
examples

In [0]:
def get_predicate_labels():
  return ['主演',
 '目',
 '身高',
 '出生日期',
 '国籍',
 '连载网站',
 '作者',
 '歌手',
 '海拔',
 '出生地',
 '导演',
 '气候',
 '朝代',
 '妻子',
 '丈夫',
 '民族',
 '毕业院校',
 '编剧',
 '出品公司',
 '父亲',
 '出版社',
 '作词',
 '作曲',
 '母亲',
 '成立日期',
 '字',
 '号',
 '所属专辑',
 '所在城市',
 '总部地点',
 '主持人',
 '上映时间',
 '首都',
 '创始人',
 '祖籍',
 '改编自',
 '制片人',
 '注册资本',
 '人口数量',
 '面积',
 '主角',
 '占地面积',
 '嘉宾',
 '简称',
 '董事长',
 '官方语言',
 '邮政编码',
 '专业代码',
 '修业年限']
def get_token_labels():
  BIO_token_labels = ["[Padding]", "[category]", "[##WordPiece]", "[CLS]", "[SEP]", "B-SUB", "I-SUB", "B-OBJ", "I-OBJ", "O"]  #id 0 --> [Paddding]
  return BIO_token_labels


In [15]:
examples[0].text_token[0:]

['查',
 '尔',
 '斯',
 '·',
 '阿',
 '兰',
 '基',
 '斯',
 '（',
 '[UNK]',
 '[UNK]',
 '）',
 '，',
 '1989',
 '年',
 '4',
 '月',
 '17',
 '日',
 '出',
 '生',
 '于',
 '智',
 '利',
 '圣',
 '地',
 '亚',
 '哥',
 '，',
 '智',
 '利',
 '职',
 '业',
 '足',
 '球',
 '运',
 '动',
 '员',
 '，',
 '司',
 '职',
 '中',
 '场',
 '，',
 '效',
 '力',
 '于',
 '德',
 '国',
 '足',
 '球',
 '甲',
 '级',
 '联',
 '赛',
 '勒',
 '沃',
 '库',
 '森',
 '足',
 '球',
 '俱',
 '乐',
 '部',
 '出生地']

In [16]:
feature = convert_single_example(0, examples[0], get_token_labels(), get_predicate_labels(), 128, tokenizer)
feature.token_label_ids

INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid:None


INFO:tensorflow:guid:None


INFO:tensorflow:tokens: [CLS]查尔斯·阿兰基斯（[UNK][UNK]），1989年4月17日出生于智利圣地亚哥，智利职业足球运动员，司职中场，效力于德国足球甲级联赛勒沃库森足球俱乐[SEP]


...

INFO:tensorflow:token_label_ids: 3 5 6 6 6 6 6 6 6 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 7 8 8 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4


INFO:tensorflow:token_label_ids: 3 5 6 6 6 6 6 6 6 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 7 8 8 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4


INFO:tensorflow:predicate_id: 9


INFO:tensorflow:predicate_id: 9


[3,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 7,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4]

In [0]:
import json
import pandas as pd
def prepare_data(input_file, limit=None):
  examples = []
  with open(input_file, 'r') as f:
    for (index, line) in enumerate(f):
      if limit and index>=limit:
        break
      if index % 10000 == 0:
        print('sample {%d}' % index)
      line_data = json.loads(line)
      spo_example = build_SPO_Example(line_data)
      for example in spo_example:
        examples.append(example)
  return examples

In [18]:
train_examples = prepare_data('/tmp/train_data.json')[0:50000]


sample {0}
sample {10000}
sample {20000}
sample {30000}
sample {40000}
sample {50000}
sample {60000}
sample {70000}
sample {80000}
sample {90000}
sample {100000}
sample {110000}
sample {120000}
sample {130000}
sample {140000}
sample {150000}
sample {160000}
sample {170000}


In [19]:
train_features = convert_examples_to_features(train_examples, get_token_labels(), get_predicate_labels(), 128, tokenizer)

INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid:None


INFO:tensorflow:guid:None


INFO:tensorflow:tokens: [CLS]如何演好自己的角色，请读《演员自我修养》《喜剧之王》周星驰崛起于穷困潦倒之中的独门秘笈[SEP][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding][Padding]


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:token_label_ids: 3 5 6 6 6 6 6 6 6 6 6 6 6 9 7 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4


INFO:tensorflow:token_label_ids: 3 5 6 6 6 6 6 6 6 6 6 6 6 9 7 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4


INFO:tensorflow:predicate_id: 3


INFO:tensorflow:predicate_id: 3


In [0]:
def input_fn_builder(input_features, seq_length, is_training, num_token_labels, num_predicate_labels, drop_remainder):
  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_token_label_ids = []
  all_predicate_label_ids = []

  for feature in input_features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_token_label_ids.append(feature.token_label_ids)
    all_predicate_label_ids.append(feature.predicate_label_id)
  
  def input_fn(params):
    batch_size = params['batch_size']
    num_examples = len(input_features)
    d = tf.data.Dataset.from_tensor_slices({
        'input_ids': tf.constant(all_input_ids, dtype=tf.int32, shape=[num_examples, seq_length]),
        'input_mask': tf.constant(all_input_mask, dtype=tf.int32, shape=[num_examples, seq_length]),
        'segment_ids': tf.constant(all_segment_ids, dtype=tf.int32, shape=[num_examples, seq_length]),
        'token_label_ids':tf.constant(all_token_label_ids, dtype=tf.int32, shape=[num_examples, seq_length]),
        'predicate_label_ids': tf.constant(all_predicate_label_ids, dtype=tf.int32, shape=[num_examples])
    })
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)
    d = d.batch(batch_size, drop_remainder=drop_remainder)
    return d
  return input_fn

In [0]:
def create_model(is_training, input_ids, input_mask, segment_ids, token_label_ids, predicate_label_ids, num_token_labels, num_predicate_labels, seq_length):
  bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
  bert_input = dict(input_ids=input_ids, segment_ids=segment_ids, input_mask=input_mask)
  bert_output = bert_module(inputs=bert_input, signature='tokens', as_dict=True)

  pooled_output_layer = bert_output['pooled_output']
  pooled_hidden_size = pooled_output_layer.shape[-1].value
  pooled_output_weight = tf.get_variable('pooled_output_weight', [num_predicate_labels, pooled_hidden_size], initializer=tf.truncated_normal_initializer(stddev=0.02))
  pooled_output_bias = tf.get_variable('pooled_output_bias', [num_predicate_labels], initializer=tf.zeros_initializer)

  with tf.variable_scope('predicate_loss'):
    if is_training:
      pooled_output_layer = tf.nn.dropout(pooled_output_layer, keep_prob=0.9)
    predicate_logits = tf.matmul(pooled_output_layer, pooled_output_weight, transpose_b=True)
    predicate_logits = tf.add(predicate_logits, pooled_output_bias)
    predicate_probabilities = tf.nn.softmax(predicate_logits, axis=-1)
    predicate_prediction = tf.argmax(predicate_probabilities, axis=-1, output_type=tf.int32)
    predicate_labels = tf.one_hot(predicate_label_ids, depth=num_predicate_labels, dtype=tf.float32)
    predicate_per_example_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=predicate_logits, labels=predicate_labels), -1)
    predicate_loss = tf.reduce_mean(predicate_per_example_loss)
    
  sequence_output_layer = bert_output['sequence_output']
  sequence_hidden_size = sequence_output_layer.shape[-1].value
  sequence_output_weight = tf.get_variable('sequence_output_weight', [num_token_labels, sequence_hidden_size], initializer=tf.truncated_normal_initializer(stddev=0.02))
  sequence_output_bias = tf.get_variable('sequence_output_bias', [num_token_labels], initializer=tf.zeros_initializer)
  
  with tf.variable_scope('token_label_loss'):
    if is_training:
      sequence_output_layer = tf.nn.dropout(sequence_output_layer, keep_prob=0.9)
    sequence_output_layer = tf.reshape(sequence_output_layer, [-1, sequence_hidden_size])
    token_label_logits = tf.matmul(sequence_output_layer, sequence_output_weight, transpose_b=True)
    token_label_logits = tf.add(token_label_logits, sequence_output_bias)
    token_label_logits = tf.reshape(token_label_logits, [-1, seq_length, num_token_labels])

    token_label_log_probs = tf.nn.log_softmax(token_label_logits, axis=-1)
    token_label_one_hot_labels = tf.one_hot(token_label_ids, depth=num_token_labels, axis=-1)
    token_label_per_example_loss = -tf.reduce_sum(token_label_one_hot_labels*token_label_log_probs, axis=-1)
    token_label_loss = tf.reduce_sum(token_label_per_example_loss)
    token_label_probabilities = tf.nn.softmax(token_label_logits, axis=-1)
    token_label_predictions = tf.argmax(token_label_probabilities, axis=-1)

  loss = 0.5 * predicate_loss + token_label_loss

  return (loss, 
          predicate_loss, predicate_per_example_loss, predicate_probabilities, predicate_prediction,
          token_label_loss, token_label_per_example_loss, token_label_logits, token_label_predictions)

In [0]:
def model_fn_builder(num_token_labels, num_predicate_labels, learning_rate, num_train_steps, num_warmup_steps, seq_length):
  def model_fn(features, labels, mode, params):
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    token_label_ids = features["token_label_ids"]
    predicate_ids = features["predicate_label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, 
     predicate_loss, predicate_per_example_loss, predicate_probabilities, predicate_prediction,
     token_label_loss, token_label_per_example_loss, token_label_logits, token_label_predictions) = create_model(is_training, 
                                                    input_ids, input_mask, segment_ids, token_label_ids, predicate_ids, num_token_labels, num_predicate_labels, seq_length)

    if is_training:
      train_op = optimization.create_optimizer(total_loss, learning_rate, num_train_steps, num_warmup_steps, False)
      return tf.estimator.EstimatorSpec(mode=mode, loss=total_loss, train_op = train_op)

    elif mode==tf.estimator.ModeKeys.EVAL:
      def metric_fn(predicate_loss, token_label_per_example_loss, predicate_probabilities, token_label_ids, token_label_logits):
        predicate_prediction = tf.argmax(predicate_probabilities, axis=-1, output_type=tf.int32)
        token_label_predictions = tf.argmax(token_label_logits, axis=-1, output_type=tf.int32)
        token_label_pos_indices_list = list(range(num_token_labels))[4:]
        pos_indices_list = token_label_pos_indices_list[:-1]

        token_label_precision_macro = tf_metrics.precision(token_label_ids, token_label_predictions, num_token_labels, pos_indices_list, average='macro')
        token_label_recall_macro = tf_metrics.recall(token_label_ids, token_label_predictions, num_token_labels, pos_indices_list, average='macro')
        token_label_f_macro = tf_metrics.f1(token_labels_ids, token_label_predictions, num_token_labels, pos_indices_list, average='macro')

        token_label_precision_micro = tf_metrics.precision(token_label_ids, token_label_predictions, num_token_labels, pos_indices_list, average='micro')
        token_label_recall_micro = tf_metrics.recall(token_label_ids, token_label_predictions, num_token_labels, average='micro')
        token_label_f_micro = tf_metrics.f1(token_label_ids, token_label_predictions, num_token_labels, pos_indices_list, average='micro')

        token_label_loss = tf.metrics.mean(values=token_label_per_example_loss)
        predicate_loss = tf.metrics.mean(values=predicate_loss)

        return {
            'eval_predicate_loss': predicate_loss, 
            'predicate_prediction': predicate_prediction,
            'eval_token_label_precision(macro)': token_label_precision_macro,
            'eval_token_label_recall(macro)': token_label_recall_macro,
            'eval_token_label_f(macro)': token_label_f_macro,
            'eval_token_label_precision(micro)': token_label_precision_micro,
            'eval_token_label_recall(micro)': token_label_recall_micro,
            'eval_token_label_f(micro)': token_label_f_micro,
            'eval_token_label_loss': token_label_loss
        }

      eval_metrics = (metric_fn, [predicate_loss, token_label_per_example_loss, predicate_probabilities, token_label_ids, token_label_logits])

      return tf.estimator.EstimatorSpec(eval_metrics=eval_metrics, mode=mode, loss=total_loss)

    else:
      """ predicting """
      predictions = {
          'predicate_probabilities': predicate_probabilities,
          'predicate_prediction': predicate_prediction,
          'token_label_predictions': token_label_predictions
      }

      return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
  return model_fn
      

In [0]:
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCH = 3.0
WARMUP_PROPORTION = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
run_config = tf.estimator.RunConfig(
    model_dir = OUTPUT_DIR,
    save_summary_steps = SAVE_SUMMARY_STEPS,
    save_checkpoints_steps = SAVE_CHECKPOINTS_STEPS
)

In [25]:

num_train_steps = int(len(train_features) / BATCH_SIZE*NUM_TRAIN_EPOCH)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = model_fn_builder(len(get_token_labels()), len(get_predicate_labels()), LEARNING_RATE, num_train_steps, num_warmup_steps, 128)
estimator = tf.estimator.Estimator(model_fn=model_fn, config=run_config, params={'batch_size': BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'gs://bert_classification/extract_training_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff167fbf0b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'gs://bert_classification/extract_training_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff167fbf0b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
train_input_fn = input_fn_builder(train_features, 128, True, len(get_token_labels()), len(get_predicate_labels()), True)

from datetime import datetime
print(f'Beginning Training')
current_time = datetime.now()
estimator.train(input_fn = train_input_fn, max_steps=num_train_steps)

Beginning Training
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:loss = 64.20862, step = 9300 (26.244 sec)


INFO:tensorflow:Saving checkpoints for 9375 into gs://bert_classification/extract_training_output/model.ckpt.


INFO:tensorflow:Saving checkpoints for 9375 into gs://bert_classification/extract_training_output/model.ckpt.


INFO:tensorflow:Loss for final step: 39.58505.


INFO:tensorflow:Loss for final step: 39.58505.


In [0]:
def get_prediction(texts_with_predicates):
  predict_examples = []
  
  for text_with_predicate in texts_with_predicates:
    text = text_with_predicate['text']
    predicate = text_with_predicate['predicate']
    text_token = tokenizer.tokenize(text)
    text_token.append(predicate)
    token_labels = ['O']*(len(text_token)-1)
    token_labels.append(predicate)
    predict_example = InputExample(guid=None, text_token=text_token, token_labels=token_labels)
    predict_examples.append(predict_example)

  predict_features = convert_examples_to_features(predict_examples, get_token_labels(), get_predicate_labels(), 128, tokenizer)
  predict_input_fn = input_fn_builder(predict_features, 128, False, len(get_token_labels()), len(get_predicate_labels()), False)
  predictions = estimator.predict(input_fn = predict_input_fn)

  results = []
  text_token_labels = get_token_labels()
  for (i, prediction) in enumerate(predictions):
    text = texts_with_predicates[i]['text']
    text_token = tokenizer.tokenize(text)
    token_label_predictions = prediction['token_label_predictions']
    subject_word = []
    object_word = []

    for (index, label) in enumerate(token_label_predictions):
      if label == text_token_labels.index('I-OBJ') or label == text_token_labels.index('B-OBJ'):
        object_word.append(text_token[index-1])
      elif label == text_token_labels.index('I-SUB') or label == text_token_labels.index('B-SUB'):
        subject_word.append(text_token[index-1])
    result = {'text': text,  'subject': subject_word, 'object': object_word, 'predicate':prediction['predicate_prediction']
              # , 'probs': prediction['token_label_predictions']
              }
    results.append(result)

  return results


In [44]:
get_token_labels()

['[Padding]',
 '[category]',
 '[##WordPiece]',
 '[CLS]',
 '[SEP]',
 'B-SUB',
 'I-SUB',
 'B-OBJ',
 'I-OBJ',
 'O']

In [63]:
texts_with_predictions = [{'text':'周佛海被捕入狱之后，其妻杨淑慧散尽家产请蒋介石枪下留人，于是周佛海从死刑变为无期，不过此人或许作恶多端，改判没多久便病逝于监狱，据悉是心脏病发作', 'predicate': '妻子'},
                          {'text':'周佛海被捕入狱之后，其妻杨淑慧散尽家产请蒋介石枪下留人，于是周佛海从死刑变为无期，不过此人或许作恶多端，改判没多久便病逝于监狱，据悉是心脏病发作', 'predicate':'丈夫'},
                          {'text':'刘灿明现居广州，意大利人，生于1985年，现在从事软件开发工作', 'predicate':'出生日期'},
                          {'text':'刘灿明现居广州，意大利人，生于1985年，现在从事软件开发工作', 'predicate':'国籍'},
                          {'text':'刘灿明现居广州，意大利人，生于1985年，现在从事软件开发工作', 'predicate':'出生地'}
                          ]

results = get_prediction(texts_with_predictions)


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid:None


INFO:tensorflow:guid:None


INFO:tensorflow:tokens: [CLS]周佛海被捕入狱之后，其妻杨淑慧散尽家产请蒋介石枪下留人，于是周佛海从死刑变为无期，不过此人或许作恶多端，改判没多久便病逝于监狱[SEP]


...

INFO:tensorflow:Restoring parameters from gs://bert_classification/extract_training_output/model.ckpt-9375


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [64]:
results

[{'object': ['杨', '淑', '慧'],
  'predicate': 13,
  'subject': ['周', '佛', '海'],
  'text': '周佛海被捕入狱之后，其妻杨淑慧散尽家产请蒋介石枪下留人，于是周佛海从死刑变为无期，不过此人或许作恶多端，改判没多久便病逝于监狱，据悉是心脏病发作'},
 {'object': ['周', '佛', '海'],
  'predicate': 14,
  'subject': ['杨', '淑', '慧'],
  'text': '周佛海被捕入狱之后，其妻杨淑慧散尽家产请蒋介石枪下留人，于是周佛海从死刑变为无期，不过此人或许作恶多端，改判没多久便病逝于监狱，据悉是心脏病发作'},
 {'object': ['1985', '年'],
  'predicate': 3,
  'subject': ['刘', '灿', '明'],
  'text': '刘灿明现居广州，意大利人，生于1985年，现在从事软件开发工作'},
 {'object': ['意', '大', '利'],
  'predicate': 4,
  'subject': ['刘', '灿', '明'],
  'text': '刘灿明现居广州，意大利人，生于1985年，现在从事软件开发工作'},
 {'object': ['广', '州'],
  'predicate': 9,
  'subject': ['刘', '灿', '明'],
  'text': '刘灿明现居广州，意大利人，生于1985年，现在从事软件开发工作'}]